In [1]:
import os

os.chdir("../")  # Укажите путь к корню проекта
print(os.getcwd())  # Проверьте, что путь изменился

c:\Users\MishaV\Desktop\karpov\ML\L22


In [1]:
import warnings
import numpy as np
import pandas as pd
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from category_encoders import OneHotEncoder
from category_encoders import TargetEncoder
from sqlalchemy import create_engine
from src.data import (
    select_data,
    CustomTransformer,
    train_test_split,
    transform_data,
    OHE_COLS,
    MTE_COLS,
)
from src.train import save_model, load_model, model_score

warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
engine = create_engine(
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
    "postgres.lab.karpov.courses:6432/startml"
)

In [4]:
# data = select_data(
#     seed=0.42,
#     count=4_800_000,
#     method="stratified",
#     users_scv=True,
#     posts_csv=True,
#     feed_data_csv=True,
#     save_path=r".\data\raw\stratified42.csv",
# )
data = pd.read_csv(r"..\data\raw\stratified42.csv", parse_dates=["timestamp"])
data["timestamp"].is_monotonic_increasing

True

In [5]:
(data["target"] == 1).sum(), (data["target"] == 0).sum()

(589259, 4369645)

In [6]:
X = data.drop("target", axis=1)
y = data["target"]

# более новые данные (по timestamp) относим к тестовой выборке
X_train, X_test, y_train, y_test = train_test_split(X, y, 0.2, is_sorted=True)
custom_cv = [(X_train.index, X_test.index)]

In [11]:
transformers_list = [
    ("te_encoder", TargetEncoder(), MTE_COLS),
    ("ohe_encoder", OneHotEncoder(), OHE_COLS),
]
column_transformer = ColumnTransformer(transformers=transformers_list)

pipe = Pipeline(
    [
        ("custom_transformer", CustomTransformer()),
        ("column_transformer", column_transformer),
        ("model", None),
    ]
)

xgboost

In [ ]:
transformers_list = [
    ("te_encoder", TargetEncoder(), MTE_COLS),
    ("ohe_encoder", OneHotEncoder(), OHE_COLS),
]
column_transformer = ColumnTransformer(transformers=transformers_list)

pipe = Pipeline(
    [
        ("custom_transformer", CustomTransformer()),
        ("column_transformer", column_transformer),
        (
            "model",
            xgb.XGBClassifier(
                random_state=42,
                use_label_encoder=False,
                n_estimators=100,
                learning_rate=0.2,
                max_depth=9,
                subsample=0.8,
                colsample_bytree=0.8,
                gamma=1,
                reg_lambda=1,
            ),
        ),
    ]
)

In [ ]:
pipe.fit(X_train, y_train)

In [14]:
y_proba = pipe.predict_proba(X_test)[:, 1]

In [ ]:
model_score(X_test, y_test, y_proba)

In [10]:
save_model("XGB", pipe)

In [6]:
xgb_pipe = load_model("XGB250112040718.pkl")

In [9]:
y_proba = xgb_pipe.predict_proba(X_test)[:, 1]

In [ ]:
model_score(X_test, y_test, y_proba)

CustomTransformer + CatBoost

In [7]:
catboost_ = CatBoostClassifier(
    random_state=42, auto_class_weights="Balanced", cat_features=OHE_COLS + MTE_COLS
)
pipe = Pipeline(
    [
        ("custom_transformer", CustomTransformer()),
        ("model", catboost_),
    ]
)

In [8]:
pipe.fit(X_train, y_train)

Learning rate set to 0.354391
0:	learn: 0.6282876	total: 4.56s	remaining: 1h 15m 59s
1:	learn: 0.6170069	total: 7.74s	remaining: 1h 4m 22s
2:	learn: 0.6127871	total: 11.2s	remaining: 1h 2m 15s
3:	learn: 0.6105943	total: 13.9s	remaining: 57m 48s
4:	learn: 0.6040499	total: 16.5s	remaining: 54m 50s
5:	learn: 0.6031328	total: 20.1s	remaining: 55m 21s
6:	learn: 0.6007620	total: 23s	remaining: 54m 21s
7:	learn: 0.5991344	total: 26.4s	remaining: 54m 29s
8:	learn: 0.5988493	total: 29.4s	remaining: 54m 1s
9:	learn: 0.5978800	total: 32.1s	remaining: 52m 57s
10:	learn: 0.5973191	total: 34.7s	remaining: 51m 59s
11:	learn: 0.5962523	total: 37.6s	remaining: 51m 33s
12:	learn: 0.5961273	total: 39.9s	remaining: 50m 30s
13:	learn: 0.5959423	total: 43.3s	remaining: 50m 46s
14:	learn: 0.5956496	total: 45.7s	remaining: 50m
15:	learn: 0.5954960	total: 48s	remaining: 49m 10s
16:	learn: 0.5952677	total: 50.8s	remaining: 48m 59s
17:	learn: 0.5948516	total: 53.6s	remaining: 48m 43s
18:	learn: 0.5942872	total: 

Pipeline(steps=[('custom_transformer', CustomTransformer()),
                ('model',
                 <catboost.core.CatBoostClassifier object at 0x0000023D2492F9A0>)])

In [9]:
y_proba = pipe.predict_proba(X_test)[:, 1]

In [10]:
model_score(X_test, y_test, y_proba)

roc_auc_score: 0.5334057473675946
average_precision_score: 0.1224969682809907
hitrate: 0.4811676082862524


In [11]:
save_model("pipe_catboost", pipe)

In [14]:
pipe.named_steps["model"]

In [20]:
pipe.named_steps["model"].save_model("../models/catboost_model.cbm")

In [21]:
model = CatBoostClassifier()
model.load_model("../models/catboost_model.cbm")

In [22]:
pipe.set_params(model=model)

Pipeline(steps=[('custom_transformer', CustomTransformer()),
                ('model',
                 <catboost.core.CatBoostClassifier object at 0x0000023D703F3F10>)])

In [23]:
y_proba = pipe.predict_proba(X_test)[:, 1]

In [24]:
model_score(X_test, y_test, y_proba)

roc_auc_score: 0.5334057473675946
average_precision_score: 0.1224969682809907
hitrate: 0.4811676082862524


Only CatBoost

In [ ]:
catboost_.fit(X_train, y_train)

In [ ]:
y_proba = catboost_.predict_proba(X_test)[:, 1]

In [ ]:
model_score(X_test, y_test, y_proba)

In [ ]:
catboost_.save_model("../models/only_cat", format="cbm")